In [28]:
import praw
import requests
import time
import pandas as pd
import numpy as np

In [22]:
reddit = praw.Reddit(
    client_id="JfSqEOisN3NJYA",
    client_secret="gTIly8BtRRIAtAFXDC49gDPR9_k",
    user_agent="my user agent")

In [41]:
def submissions_pushshift_praw(subreddit, start=None, end=None, limit=100, extra_query=""):
    """
    A simple function that returns a list of PRAW submission objects during a particular period from a defined sub.
    This function serves as a replacement for the now deprecated PRAW `submissions()` method.
    
    :param subreddit: A subreddit name to fetch submissions from.
    :param start: A Unix time integer. Posts fetched will be AFTER this time. (default: None)
    :param end: A Unix time integer. Posts fetched will be BEFORE this time. (default: None)
    :param limit: There needs to be a defined limit of results (default: 100), or Pushshift will return only 25.
    :param extra_query: A query string is optional. If an extra_query string is not supplied, 
                        the function will just grab everything from the defined time period. (default: empty string)
    
    Submissions are yielded newest first.
    
    For more information on PRAW, see: https://github.com/praw-dev/praw 
    For more information on Pushshift, see: https://github.com/pushshift/api
    """
    matching_praw_submissions = []
    
    # Default time values if none are defined (credit to u/bboe's PRAW `submissions()` for this section)
    utc_offset = 0
    now = int(time.time())
    start = max(int(start) + utc_offset if start else 0, 0)
    end = min(int(end) if end else now, now) + utc_offset
    
    # Format our search link properly.
    search_link = ('https://api.pushshift.io/reddit/submission/search/'
                   '?subreddit={}&after={}&before={}&sort_type=score&sort=asc&limit={}&q={}')
    search_link = search_link.format(subreddit, start, end, limit, extra_query)
    
    # Get the data from Pushshift as JSON.
    retrieved_data = requests.get(search_link)
    returned_submissions = retrieved_data.json()['data']
    
    # Iterate over the returned submissions to convert them to PRAW submission objects.
    for submission in returned_submissions:
        
        # Take the ID, fetch the PRAW submission object, and append to our list
        praw_submission = reddit.submission(id=submission['id'])
        matching_praw_submissions.append(praw_submission)
     
    # Return all PRAW submissions that were obtained.
    return matching_praw_submissions

In [81]:
reddit_posts = {'source': [], 'date': [], 'post title': [], 'upvotes': [], 'company': []}

In [82]:
day = 0
for j in range(1593561600,1601510400,86400):
    day += 1
    print(day)
    start = j
    end = j+86400
    for i in submissions_pushshift_praw('investing', start, end, limit = 100):
        reddit_posts['source'].append('Reddit')
        reddit_posts['date'].append(i.created_utc)
        reddit_posts['post title'].append(i.title)
        reddit_posts['upvotes'].append(i.score)
        reddit_posts['company'].append('NA')

1
2
3
4


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [67]:
reddit_df = pd.DataFrame(reddit_posts)

In [68]:
reddit_df

,source,date,post title,upvotes,company
0,Reddit,1.593615e+09,Rate my Stock Portfolio - Female Investor,0,NA
1,Reddit,1.593709e+09,[question] Where can i see the total of actual...,1,NA
2,Reddit,1.593794e+09,MARK -Remark Holdings,0,NA
3,Reddit,1.593891e+09,Should I invest in American stocks? Me being C...,2,NA
4,Reddit,1.593966e+09,Advice for long term investing,1,NA
5,Reddit,1.594047e+09,Is IBM a good buy right now?,0,NA
6,Reddit,1.594127e+09,A penny stock just got approval from patent of...,0,NA
